Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [464]:
from random import choices,randint, random
from collections import namedtuple
import logging

import lab9_lib

In [465]:
NUM_LOCI = 1000
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
INSTANCES = 10

NUM_GENERATIONS = 100

In [466]:
fitness = lab9_lib.make_problem(INSTANCES)
for n in range(POPULATION_SIZE):
    ind = choices([0, 1], k=NUM_LOCI)
    # print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}" if NUM_LOCI<=10 else f"ind {n}: {fitness(ind):.2%}")

print(fitness.calls)

0


Initialize population

In [467]:
class Individual:
    def __init__(self, genome, fitness):
        self.genome = genome
        self.fitness = fitness

def initialize_population(POPULATION_SIZE, NUM_LOCI):
    population = []

    for _ in range(POPULATION_SIZE):
        genome = tuple(choices([0, 1], k=NUM_LOCI))
        Fitness = fitness(genome)
        individual = Individual(genome=genome, fitness=Fitness)
        population.append(individual)

    return population


Cross over and mutation functions

In [468]:
def single_point_crossover(genome1, genome2):
    point = randint(0, len(genome1))
    return genome1[:point] + genome2[point:]
    # return genome1[:point] + genome2[point:], genome2[:point] + genome1[point:]


def multiple_point_crossover(genome1, genome2, num_points):
    points = [randint(0, len(genome1)) for _ in range(num_points)]
    points.sort()

    offspring = []
    last_point = 0

    for point in points:
        offspring += genome1[last_point:point]
        genome1, genome2 = genome2, genome1  # Swap the parents
        last_point = point
    offspring += genome1[last_point:]
    return offspring


# def mutate(ind: Individual) -> Individual:
#     num = randint(0, NUM_LOCI - 1)
#     ind.genome[num] = not ind.genome[num]
#     return ind

def mutate(genome):
    point = randint(0, NUM_LOCI - 1)
    return genome[:point] + (1 - genome[point],) + genome[point + 1 :]

def tournament(population, size=2):
    selected = choices(population, k=size)
    return max(selected, key=lambda j:j.fitness)

In [469]:
# generations = 0
# population = initialize_population(POPULATION_SIZE, NUM_LOCI)
# while population[0].fitness < 1:
#     generations += 1
#     offspring = list()
#     for o in range(OFFSPRING_SIZE):
#         p1 = tournament(population)
#         mutated_genome = None
#         if random() < 0.2:
#             mutated_genome = mutate(p1.genome)
#         else:
#             p2 = tournament(population)
#             crossovered_genome = single_point_crossover(p1.genome, p2.genome)
            
#         offspring.append(Individual(mutated_genome, fitness(mutated_genome)))
#     population.extend(offspring)
#     population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
# logging.info(f"ga: Probelm solved in {generations:,} generations")


In [470]:
def ss_GA(population, offspring, size):
    """
    steady-state GA: from x parents we obtain y children and from x+y individuals we take only x ones
    """
    
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:size]

    return population

#algorithm based on the professor's repository

In [471]:

selection = ss_GA
best_fitness = 0
counter = 0

for g in range(NUM_GENERATIONS):
    offspring = list()
    currentfitness = 0

    for i in range(OFFSPRING_SIZE):
        if random() < 0.3:
            p = tournament(population)
            o = mutate(p.genome)
        else:
            p1 = tournament(population)
            p2 = tournament(population)
            o = single_point_crossover(p1.genome, p2.genome)
        f = fitness(o)

        if f > currentfitness: 
            currentfitness = f
        offspring.append(Individual(o, f))
    population = selection(population, offspring, POPULATION_SIZE)
    
    if currentfitness <= best_fitness:
        counter += 1
        
        if counter >= 100:
            print("break at generation n.", g, "fitness: ", best_fitness)
            break
        
    elif currentfitness > best_fitness:
        counter = 0

        best_fitness = currentfitness
        if best_fitness == 1:
            print("break at generation n.", g, "fitness: ", best_fitness)
            break
    
    print("Generation",g ,"counter:", counter, "best_fitness:", best_fitness, "difference:",best_fitness-currentfitness)

population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]

# print(fitness.calls)


Generation 0 counter: 0 best_fitness: 0.06790001357900001 difference: 0.0
Generation 1 counter: 1 best_fitness: 0.06790001357900001 difference: 0.0008999021100000032
Generation 2 counter: 0 best_fitness: 0.06800011246900001 difference: 0.0
Generation 3 counter: 0 best_fitness: 0.13801111469 difference: 0.0
Generation 4 counter: 1 best_fitness: 0.13801111469 difference: 0.070111002221
Generation 5 counter: 2 best_fitness: 0.13801111469 difference: 0.07101100312099999
Generation 6 counter: 3 best_fitness: 0.13801111469 difference: 0.07012110312099999
Generation 7 counter: 4 best_fitness: 0.13801111469 difference: 0.07001100212099999
Generation 8 counter: 5 best_fitness: 0.13801111469 difference: 0.07001100312099999
Generation 9 counter: 6 best_fitness: 0.13801111469 difference: 9.990099999995783e-06
Generation 10 counter: 7 best_fitness: 0.13801111469 difference: 0.07001100112099999
Generation 11 counter: 8 best_fitness: 0.13801111469 difference: 0.07101100312099999
Generation 12 counter

Generation 41 counter: 38 best_fitness: 0.13801111469 difference: 9.969999999998036e-06
Generation 42 counter: 39 best_fitness: 0.13801111469 difference: 9.980100000001046e-06
Generation 43 counter: 40 best_fitness: 0.13801111469 difference: 9.978999999993299e-06
Generation 44 counter: 41 best_fitness: 0.13801111469 difference: 0.070111102321
Generation 45 counter: 42 best_fitness: 0.13801111469 difference: 0.07001100222099998
Generation 46 counter: 43 best_fitness: 0.13801111469 difference: 1.0968899999991288e-05
Generation 47 counter: 44 best_fitness: 0.13801111469 difference: 0.07101100222099999
Generation 48 counter: 45 best_fitness: 0.13801111469 difference: 0.07001000222099998
Generation 49 counter: 46 best_fitness: 0.13801111469 difference: 0.07001100222199999
Generation 50 counter: 47 best_fitness: 0.13801111469 difference: 0.070011002211
Generation 51 counter: 48 best_fitness: 0.13801111469 difference: 0.07001100222
Generation 52 counter: 49 best_fitness: 0.13801111469 differe

In [472]:
# generations = 0
# population = initialize_population(POPULATION_SIZE, NUM_LOCI)
# while population[0].fitness < 1:
#     generations += 1
#     offspring = list()
#     for o in range(OFFSPRING_SIZE):
#         p1 = tournament(population)
#         mutated_genome = None
#         if random() < 0.2:
#             mutated_genome = mutate(p1.genome)
#         else:
#             p2 = tournament(population)
#             crossovered_genome = single_point_crossover(p1.genome, p2.genome)
            
#         offspring.append(Individual(mutated_genome, fitness(mutated_genome)))
#     population.extend(offspring)
#     population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
# logging.info(f"ga: Probelm solved in {generations:,} generations")


In [473]:


# # Example usage
# PROBLEM_SIZE = 10
# genome1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
# genome2 = [11, 12, 13, 14, 15, 16, 17, 18]

# num_points = 4
# result_multiple_cut = multiple_point_crossover(genome1, genome2, num_points)

# print("Multiple Cut Result:", result_multiple_cut)
